# Importación de librerias

In [1]:
from scipy.spatial.distance import cityblock, euclidean
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import OneClassSVM
from sklearn.mixture import GaussianMixture
from scipy.spatial import distance

# Lectura de archivo

In [2]:
path = "./data/cmu/keystroke.csv" 
df = pd.read_csv(path)
df.drop(['sessionIndex', 'rep'], axis=1, inplace=True)
subjects = df["subject"].unique()

# Metodo de distancias

In [3]:
def euclideanDistance(a, b):
    return distance.euclidean(a.values, b.values)
    #return np.linalg.norm(a.values - b.values)

def manhattanDistance(a, b):
    return distance.cityblock(a.values, b.values)

def cosDistance(a,b):
    return distance.cosine(a.values, b.values)

#Distancias a calcular
distancesMethods = [1, 2, 3]

In [4]:
df

,subject,H.period,DD.period.t,UD.period.t,H.t,DD.t.i,UD.t.i,H.i,DD.i.e,UD.i.e,...,H.a,DD.a.n,UD.a.n,H.n,DD.n.l,UD.n.l,H.l,DD.l.Return,UD.l.Return,H.Return
0,s002,0.1491,0.3979,0.2488,0.1069,0.1674,0.0605,0.1169,0.2212,0.1043,...,0.1349,0.1484,0.0135,0.0932,0.3515,0.2583,0.1338,0.3509,0.2171,0.0742
1,s002,0.1111,0.3451,0.2340,0.0694,0.1283,0.0589,0.0908,0.1357,0.0449,...,0.1412,0.2558,0.1146,0.1146,0.2642,0.1496,0.0839,0.2756,0.1917,0.0747
2,s002,0.1328,0.2072,0.0744,0.0731,0.1291,0.0560,0.0821,0.1542,0.0721,...,0.1621,0.2332,0.0711,0.1172,0.2705,0.1533,0.1085,0.2847,0.1762,0.0945
3,s002,0.1291,0.2515,0.1224,0.1059,0.2495,0.1436,0.1040,0.2038,0.0998,...,0.1457,0.1629,0.0172,0.0866,0.2341,0.1475,0.0845,0.3232,0.2387,0.0813
4,s002,0.1249,0.2317,0.1068,0.0895,0.1676,0.0781,0.0903,0.1589,0.0686,...,0.1312,0.1582,0.0270,0.0884,0.2517,0.1633,0.0903,0.2517,0.1614,0.0818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20395,s057,0.0884,0.0685,-0.0199,0.1095,0.1290,0.0195,0.0945,0.0757,-0.0188,...,0.1219,0.1383,0.0164,0.0820,0.1329,0.0509,0.1005,0.2054,0.1049,0.1047
20396,s057,0.0655,0.0630,-0.0025,0.0910,0.1148,0.0238,0.0916,0.0636,-0.0280,...,0.1008,0.0512,-0.0496,0.1037,0.0868,-0.0169,0.1445,0.2206,0.0761,0.1198
20397,s057,0.0939,0.1189,0.0250,0.1008,0.1122,0.0114,0.0721,0.0462,-0.0259,...,0.0913,0.1169,0.0256,0.0689,0.1311,0.0622,0.1034,0.2017,0.0983,0.0905
20398,s057,0.0923,0.1294,0.0371,0.0913,0.0990,0.0077,0.0992,0.0897,-0.0095,...,0.0882,0.0821,-0.0061,0.0576,0.0697,0.0121,0.0979,0.1917,0.0938,0.0931


# Separación de la data de entrenamiento y de prueba
Se toma de forma aleatoria el 80% de los registros de cada usuario para considerarlos como data de entrenamiento, mientras que el restante forma a pasar a la data de prueba

In [5]:
train_users = []
test_users = []
for subject in subjects:
    genuine_user_data = df.loc[df["subject"] == subject, :]
            
    imposter_data = df.loc[df["subject"] != subject, :]

    train, test_genuine  = train_test_split(genuine_user_data, train_size = 0.8, random_state=43, shuffle=True)

    train_users.append(train)
    test_users.append(test_genuine)
    

train_users = pd.concat(train_users)
test_users = pd.concat(test_users)

# Cálculo de las distancias, evaluación y resultados
1. Se calcula la distancias entre cada vector del dataset del train con el vector del test
2. Se escoge el vector individidual del train con la menor distancia encontrada
3. Se chequea si el vector individidual corresponde con el usuario del test 
3. Se comienza a contar las veces que se acierta el usuario y las veces que no
5. Se imprime los resultados

In [6]:
for distancesMethod in distancesMethods:
    countGood = 0
    countFalse = 0
    
    for i in range(test_users.shape[0]):
        value = []
        for j in range(train_users.shape[0]):
            if 1 == distancesMethod:
                value.append(euclideanDistance(train_users.iloc[j, 1:],  test_users.iloc[i, 1:]))
            elif 2 == distancesMethod:
                value.append(manhattanDistance(train_users.iloc[j, 1:],  test_users.iloc[i, 1:]))
            elif 3 == distancesMethod:
                value.append(cosDistance(train_users.iloc[j, 1:],  test_users.iloc[i, 1:]))
        idx = np.argmin(value)
        
        if train_users.iloc[idx]["subject"] == test_users.iloc[i]["subject"] :
            countGood += 1
            #print( "raw "+ str(i)+ " Total: "+str(countGood) + " / " + str(test_users.shape[0]) + "==> " +str(round(countGood*100/test_users.shape[0],2)) + "%")
        else:
            countFalse += 1
            #print( "raw "+ str(i)+ " false")
            
    print("----------------------------------------------------")
    if 1 == distancesMethod:
        print("Resumen distancia euclidiana:")
    elif 2 == distancesMethod:
        print("Resumen distancia Manhattan:")
    elif 3 == distancesMethod:
        print("Resumen distancia Coseno:")
        
    print("Total acertados "+str(countGood) + " / " + str(test_users.shape[0]) + "==> " +str(round(countGood*100/test_users.shape[0],2)) + "%")
    print("Total equivocados "+str(countFalse) + " / " + str(test_users.shape[0]) +"==> " +str(round(countFalse*100/test_users.shape[0],2)) + "%")

----------------------------------------------------
Resumen distancia euclidiana:
Total acertados 3064 / 4080==> 75.1%
Total equivocados 1016 / 4080==> 24.9%
----------------------------------------------------
Resumen distancia Manhattan:
Total acertados 3322 / 4080==> 81.42%
Total equivocados 758 / 4080==> 18.58%
----------------------------------------------------
Resumen distancia Coseno:
Total acertados 2963 / 4080==> 72.62%
Total equivocados 1117 / 4080==> 27.38%
